#### Set up libaries

In [ ]:
%matplotlib inline
import sys, os
import matplotlib.pyplot as plt
sys.path.append('./u-dales/tools/postprocessing')
import postdales as da

#### _Specify data path:_

In [ ]:
datadir = './outputs/300/'

#### Read in netcdf files

In [ ]:
# get all netcdf files
ncpaths = da.nc.getncpath(datadir)
# get netcdf fields
ncfields = da.nc.getncdata(ncpaths)
# list all avaiable variables
da.nc.variables_available(ncfields)

#### Read in coordinates

In [ ]:
ncfield = da.nc.select_ncfield(ncfields, 'fielddump')
coordinates = da.nc.get_coordinates(ncfield)
print(coordinates)

#### Read in parameters

In [ ]:
filename = 'namoptions.300'
parameterfile = os.path.join(datadir, filename)
namelist = da.inputs.parameters(parameterfile)
parameters = da.inputs.extract_f90namelist(namelist)
# print(params)

#### Read in blocks

In [ ]:
filename = 'blocks.inp.300'
blocksfile = os.path.join(datadir, filename)
blocks = da.inputs.blocks(blocksfile)
print(blocks)

#### Dimensionalise blocks

In [ ]:
# mid cell coordinates
xaxis = coordinates['xm']
yaxis = coordinates['ym']
zaxis = coordinates['zm']
dimblocks = da.data.getdimblocks(blocks, xaxis, yaxis, zaxis)
print(dimblocks)

#### Plot blocks layout

In [ ]:
limits = [xaxis[0], xaxis[-1], yaxis[0], yaxis[-1], zaxis[0], zaxis[-1]]
da.plot.blocks3d(dimblocks, limits=limits)

## Exp object for data handling

#### Create new Exp object

In [ ]:
exp = da.experiments.Exp('300')

#### Add dimensional blocks

In [ ]:
exp.add('blocks', dimblocks)

#### Add parameters from namoptions

In [ ]:
for key, val in parameters.items():
    exp.add(key, val, 'parameters')

#### Add coordinate axes

In [ ]:
for key, val in coordinates.items():
    exp.add(key, val, 'data')

#### Read in and store some variables

In [ ]:
fieldname = 'xytdump'
ncfield = da.nc.select_ncfield(ncfields, fieldname)
select_variables = {'uav' : 'uxyt', 'turbflux' : 'upwpxyt'}
for varname, varref in select_variables.items():
    var = da.nc.select_variable(varref, ncfield)
    exp.add(varname, var, 'data')

In [ ]:
exp.show(level='keys')

---
## Save data

#### Delete large netCDFs from exp and convert small ones to numpy arrays. Then save the data

In [ ]:
exp = da.experiments.netcdf_handling(exp, maxdim=2)
exp.save('./outputs/300/exp.pkl')

#### Load saved exp objects

In [ ]:
expsaved = da.experiments.load_saved_exp('./outputs/300/exp.pkl')
expsaved.show('content')